<a href="https://colab.research.google.com/github/sabyasm/manu_data_competition/blob/master/01_generate_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download & Install pytorch GPU version

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58a34000 @  0x7f51c9d742a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [34]:
import os
import os.path
import re
from collections import defaultdict

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import xgboost as xgb
from nltk.tokenize import word_tokenize
from numpy.random import RandomState
import pickle

from sklearn import datasets, linear_model
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score as f1
from sklearn.model_selection import train_test_split
from google.colab import files

import tensorflow_hub as hub

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Download RAW test and train files (I've loaded them in aws )

In [3]:
!mkdir dataset
!curl -Lo dataset/Competition_Train_Data.txt https://s3-us-west-2.amazonaws.com/manu00/Competition_Train_Data.txt
!curl -Lo dataset/Competition_Test_Data.txt https://s3-us-west-2.amazonaws.com/manu00/Competition_Test_Data.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1102k  100 1102k    0     0   670k      0  0:00:01  0:00:01 --:--:--  670k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  274k  100  274k    0     0   278k      0 --:--:-- --:--:-- --:--:--  278k


# Infersent with GloVe Embedding (let's call it v1)

In [4]:
!curl -Lo models.py https://raw.githubusercontent.com/facebookresearch/InferSent/master/models.py
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://s3.amazonaws.com/senteval/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://s3.amazonaws.com/senteval/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29737  100 29737    0     0  85697      0 --:--:-- --:--:-- --:--:-- 85697
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  10.9M      0  0:00:13  0:00:13 --:--:-- 12.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  13.0M      0  0:00:11  0:00:11 --:--:-- 16.1M


# 2GB GloVe embedding Incoming

In [5]:
!mkdir dataset
!mkdir dataset/GloVe
!curl -Lo dataset/GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip dataset/GloVe/glove.840B.300d.zip -d dataset/GloVe/

mkdir: cannot create directory ‘dataset’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  5915k      0  0:05:59  0:05:59 --:--:-- 5990k
Archive:  dataset/GloVe/glove.840B.300d.zip
  inflating: dataset/GloVe/glove.840B.300d.txt  


In [6]:
from models import InferSent
V = 1
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

# Keep it on CPU or put it on GPU
use_cuda = True
infersent = infersent.cuda() if use_cuda else infersent

# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
model_version = V
W2V_PATH = 'dataset/GloVe/glove.840B.300d.txt' if model_version == 1 else 'dataset/fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)
infersent.build_vocab_k_words(K=500000) #load 50k most frequent word

Vocab size : 500000


# ** FastText Common Crawl - 1.5G Embedding - let's call Infersent V2**

In [7]:
!mkdir dataset
!mkdir dataset/fastText
!curl -Lo dataset/fastText/crawl-300d-2M.vec.zip https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip
!unzip dataset/fastText/crawl-300d-2M.vec.zip -d dataset/fastText/

mkdir: cannot create directory ‘dataset’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  16.3M      0  0:01:28  0:01:28 --:--:-- 21.7M
Archive:  dataset/fastText/crawl-300d-2M.vec.zip
  inflating: dataset/fastText/crawl-300d-2M.vec  


In [8]:
from models import InferSent
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent2 = InferSent(params_model)
infersent2.load_state_dict(torch.load(MODEL_PATH))

# Keep it on CPU or put it on GPU
use_cuda = True
infersent2 = infersent2.cuda() if use_cuda else infersent2

# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
model_version = V
W2V_PATH = 'dataset/GloVe/glove.840B.300d.txt' if model_version == 1 else 'dataset/fastText/crawl-300d-2M.vec'
infersent2.set_w2v_path(W2V_PATH)
infersent2.build_vocab_k_words(K=500000)

Vocab size : 500000


# **Let's try Google's Universal Sentense Encoder (Transfer) - [link](https://tfhub.dev/google/universal-sentence-encoder-large/3)**

# **USE V3 model load**

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [10]:
embed = hub.Module(module_url)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.


# **Download common crawl vector in Spacy specific format (1.5G - again)**

In [22]:
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip
!python -m spacy init-model en /tmp/fasttext-vectors --vectors-loc crawl-300d-2M.vec.zip

--2018-11-06 18:51:59--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.20.97
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.20.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  21.9MB/s    in 71s     

2018-11-06 18:53:11 (20.5 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Reading vectors from crawl-300d-2M.vec.zip
Open loc
tcmalloc: large alloc 2400002048 bytes == 0x33ee000 @  0x7f6a9a5d5001 0x7f6a98349b85 0x7f6a983acb43 0x7f6a983aea86 0x7f6a98446868 0x511b75 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x586ddd 0x59cf9e 0x4f6903 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x4ffa2a 0x5117df 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338
1999

In [23]:
#Code for obtaining sentence vector from spacy module
!python -m spacy download en
import spacy
nlp_ft = spacy.load('en')

    100% |████████████████████████████████| 37.4MB 2.7MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
def generate_spacy_cc_similarity():
  i = 0
  trainfts = []
  testfts =[]
  while i < len(traintext[0]):
    doc1 = nlp_ft(traintext[0][i])
    doc2 = nlp_ft(traintext[1][i])
    trainfts.append(doc2.similarity(doc1))
    i = i+1
  print("done")
  i = 0
  while i < len(testtext[0]):
    doc1 = nlp_ft(testtext[0][i])
    doc2 = nlp_ft(testtext[1][i])
    testfts.append(doc2.similarity(doc1))
    i = i+1
    #print(" "+str(i))
  
  return trainfts, testfts
  

# **Basic text features like count of common word etc. **

In [0]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [0]:
def feats(A, B):
    """
    Compute additional features (similar to Socher et al.)
    These alone should give the same result from their paper (~73.2 Acc)
    """
    tA = [t.split() for t in A]
    tB = [t.split() for t in B]
    
    nA = [[w for w in t if is_number(w)] for t in tA]
    nB = [[w for w in t if is_number(w)] for t in tB]

    features = np.zeros((len(A), 6))

    # n1
    for i in range(len(A)):
        if set(nA[i]) == set(nB[i]):
            features[i,0] = 1.

    # n2
    for i in range(len(A)):
        if set(nA[i]) == set(nB[i]) and len(nA[i]) > 0:
            features[i,1] = 1.

    # n3
    for i in range(len(A)):
        if set(nA[i]) <= set(nB[i]) or set(nB[i]) <= set(nA[i]): 
            features[i,2] = 1.

    # n4
    for i in range(len(A)):
        features[i,3] = 1.0 * len(set(tA[i]) & set(tB[i])) / len(set(tA[i]))

    # n5
    for i in range(len(A)):
        features[i,4] = 1.0 * len(set(tA[i]) & set(tB[i])) / len(set(tB[i]))

    # n6
    for i in range(len(A)):
        features[i,5] = 0.5 * ((1.0*len(tA[i]) / len(tB[i])) + (1.0*len(tB[i]) / len(tA[i])))

    return features

In [0]:
def generate_sentense_fts(modelname, loc='./dataset/'):
    """
    """
    print ('Preparing data...')
    traintext, testtext, labels, uid = load_data(loc)

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)
    train_features = []
    test_features = []
    
    if modelname == "infersent":
        print ('Computing ' + modelname + 'embedding for train data...')

        trainA = infersent.encode(traintext[0], bsize=128, tokenize=False, verbose=True)
        trainB = infersent.encode(traintext[1], bsize=128, tokenize=False, verbose=True)

        print ('Computing ' + modelname + 'embedding for test data...')
        
        testA = infersent.encode(testtext[0], bsize=128, tokenize=False, verbose=True)
        testB = infersent.encode(testtext[1], bsize=128, tokenize=False, verbose=True)

    if modelname == "infersent2":
        print ('Computing ' + modelname + 'embedding for train data...')

        trainA = infersent2.encode(traintext[0], bsize=128, tokenize=False, verbose=True)
        trainB = infersent2.encode(traintext[1], bsize=128, tokenize=False, verbose=True)

        print ('Computing ' + modelname + 'embedding for test data...')
        
        testA = infersent2.encode(testtext[0], bsize=128, tokenize=False, verbose=True)
        testB = infersent2.encode(testtext[1], bsize=128, tokenize=False, verbose=True)

            
        
        
    if modelname =="use_v3":
        
        print ('Computing training USE encoding...')
        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            trainA = session.run(embed(traintext[0]))
            trainB = session.run(embed(traintext[1]))
            testA = session.run(embed(testtext[0]))
            testB = session.run(embed(testtext[1]))
    
    train_features = np.c_[np.abs(trainA - trainB), trainA * trainB]
    test_features = np.c_[np.abs(testA - testB), testA * testB]
        
    
    return train_features, test_features
        

In [0]:
def load_data(loc='./dataset/'):
    """
    Load test and train dataset
    """
    trainloc = os.path.join(loc, 'Competition_Train_Data.txt')
    testloc = os.path.join(loc, 'Competition_Test_Data.txt')
    
    print(trainloc)

    trainA, trainB, testA, testB = [],[],[],[]
    trainS, testU, testS = [],[],[]

    f = open(trainloc, 'r')
    for line in f:
        text = line.strip().split('\t')
        trainA.append(' '.join(word_tokenize(text[2])))
        trainB.append(' '.join(word_tokenize(text[3])))
        trainS.append(text[0])
    f.close()
    f = open(testloc, 'r')
    for line in f:
        text = line.strip().split('\t')
        testA.append(' '.join(word_tokenize(text[1])))
        testB.append(' '.join(word_tokenize(text[2])))
        #testS.append(text[3])
        testU.append(text[0])
    f.close()
    
   
    trainS = [int(s) for s in trainS[1:]]
    testU = [int(s) for s in testU[1:]]

    return [trainA[1:], trainB[1:]], [testA[1:], testB[1:]], [trainS, testS], testU

In [15]:
traintext, testtext, labels, uid = load_data()
fts_train_1, fts_test_1 = generate_sentense_fts(modelname = 'use_v3')
fts_train_2, fts_test_2 = generate_sentense_fts(modelname = 'infersent')
fts_train_3, fts_test_3 = generate_sentense_fts(modelname = 'infersent2')

./dataset/Competition_Train_Data.txt
Preparing data...
./dataset/Competition_Train_Data.txt
Computing training USE encoding...
Preparing data...
./dataset/Competition_Train_Data.txt
Computing infersentembedding for train data...
Nb words kept : 110074/111322 (98.9%)
Speed : 929.6 sentences/s (gpu mode, bsize=128)
Nb words kept : 109905/111146 (98.9%)
Speed : 968.6 sentences/s (gpu mode, bsize=128)
Computing infersentembedding for test data...
Nb words kept : 27576/27876 (98.9%)
Speed : 903.4 sentences/s (gpu mode, bsize=128)
Nb words kept : 27606/27912 (98.9%)
Speed : 922.0 sentences/s (gpu mode, bsize=128)
Preparing data...
./dataset/Competition_Train_Data.txt
Computing infersent2embedding for train data...
Nb words kept : 107610/111322 (96.7%)
Speed : 898.3 sentences/s (gpu mode, bsize=128)
Nb words kept : 107320/111146 (96.6%)
Speed : 906.8 sentences/s (gpu mode, bsize=128)
Computing infersent2embedding for test data...
Nb words kept : 26846/27876 (96.3%)
Speed : 836.8 sentences/s (

In [27]:
%%time
trainfts, testfts = generate_spacy_cc_similarity()

done
CPU times: user 5min 45s, sys: 2min, total: 7min 45s
Wall time: 3min 56s


In [16]:
fts_train_1.shape

(4640, 1024)

In [17]:
fts_train_2.shape

(4640, 8192)

In [18]:
fts_train_3.shape

(4640, 8192)

# Combine all the features into a gigantic feature matrix

In [0]:
train_features =np.c_[fts_train_1,fts_train_2,fts_train_3,feats(traintext[0], traintext[1]),trainfts]
test_features = np.c_[fts_test_1,fts_test_2,fts_test_3,feats(testtext[0], testtext[1]),testfts]

In [33]:
train_features.shape

(4640, 17415)

# Authenticate Colab to use google drive to store the pickle file

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Save the features and labels into pickle file

In [0]:
file = open('final_features.pkl','wb')

pickle.dump(train_features, file)
pickle.dump(test_features, file)
pickle.dump(labels[0], file)
pickle.dump(labels[1], file)
pickle.dump(uid, file)

file.close()

# **Upload the feature matrix to Google Drive for modelling ~800 MB**

In [0]:
upload = drive.CreateFile({'title': 'final_features.pkl'})
upload.SetContentFile('final_features.pkl')
upload.Upload()


Go to Google Drive, select the pkl file Click Get shareable link.
Next to "Anyone with the link," click the Down arrow Down.
Click More and then On - Public on the web.
Click Save.
Click Done.


In [0]:

# https://drive.google.com/open?id=1xGT1h3XgWEmZTEkDPMpKIyK7odRNCqy6
# https://drive.google.com/file/d/1xGT1h3XgWEmZTEkDPMpKIyK7odRNCqy6/view?usp=sharing

In [0]:
#to download again
#download = drive.CreateFile({'id': '1PVGVj-BHu5I3ANnEfG4_JKn6RgyzeW3d'})
#download.GetContentFile('features.pkl')

In [41]:
!ls -ltr

total 2277412
-rw-r--r-- 1 root root 1523785255 May 24 11:52 crawl-300d-2M.vec.zip
drwxr-xr-x 2 root root       4096 Nov  5 19:58 sample_data
-rw-r--r-- 1 root root      29737 Nov  6 18:21 models.py
drwxr-xr-x 2 root root       4096 Nov  6 18:22 encoder
drwxr-xr-x 2 root root       4096 Nov  6 18:30 __pycache__
drwxr-xr-x 4 root root       4096 Nov  6 18:31 dataset
-rw-r--r-- 1 root root       2551 Nov  6 18:48 adc.json
drwx------ 3 root root       4096 Nov  6 18:49 gdrive
-rw-r--r-- 1 root root  808210640 Nov  6 19:20 final_features.pkl
